### Prepare Your Model
Ensure the model is trained and saved in an HDF5 format. Load the model using Tensorflow/Keras

In [2]:
%pip install -r requirements.txt

In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the trained model
model_path = 'trymodel.h5'
model = load_model(model_path)


### Step 2: Extract Model Information

Extract layer structures, names, configurations, and weights, and represent this information in a JSON format

In [4]:
import json
import numpy as np

def extract_model_info(model):
    model_info = {
        'layers': []
    }
    for layer in model.layers:
        layer_info = {
            'name': layer.name,
            'class_name': layer.__class__.__name__,
            'config': layer.get_config(),
            'weights': [w.tolist() for w in layer.get_weights()]
        }
        model_info['layers'].append(layer_info)
    return model_info

# Assuming `model` is already defined and loaded
model_info = extract_model_info(model)

# Save model information to a JSON file
with open('model_info.json', 'w') as f:
    json.dump(model_info, f, indent=4)


### Apply Obfuscation Strategies
Renaming Layers

In [5]:
import random
import string

def random_string(length=10):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

def obfuscate_layer_names(model_info):
    for layer in model_info['layers']:
        layer['name'] = random_string()
    return model_info

model_info = obfuscate_layer_names(model_info)


### Parameter Encapsulation

In [6]:
# def encapsulate_parameters(model_info):
#     for layer in model_info['layers']:
#         layer['weights'] = [weights.tolist() for weights in layer['weights']]
#     return model_info

# model_info = encapsulate_parameters(model_info)

import numpy as np

def encapsulate_parameters(model_info):
    for layer in model_info['layers']:
        layer['weights'] = [weights.tolist() if isinstance(weights, np.ndarray) else weights for weights in layer['weights']]
    return model_info

# # Example usage
# model_info = {
#     'layers': [
#         {'weights': [np.array([1, 2, 3]), np.array([4, 5, 6])]},
#         {'weights': [np.array([7, 8, 9]), [10, 11, 12]]}  # Mixed with a list
#     ]
# }

model_info = encapsulate_parameters(model_info)
# print(model_info)


### Neural Structure Obfuscation
- Shortcut Injection

In [7]:
from tensorflow.keras.layers import Add

def add_shortcuts(model_info):
    # Example: Add shortcuts after every layer (this is just a simplified example)
    for i in range(len(model_info['layers']) - 1):
        shortcut_layer = {
            'name': random_string(),
            'class_name': 'Add',
            'config': {},
            'weights': []
        }
        model_info['layers'].insert(i + 1, shortcut_layer)
    return model_info

model_info = add_shortcuts(model_info)


### Extra Layer Injection

In [8]:
from tensorflow.keras.layers import Dense

def inject_extra_layers(model_info):
    # Example: Inject a Dense layer with random units after every layer
    for i in range(len(model_info['layers']) - 1):
        extra_layer = {
            'name': random_string(),
            'class_name': 'Dense',
            'config': {'units': random.randint(1, 100), 'activation': 'relu'},
            'weights': []
        }
        model_info['layers'].insert(i + 1, extra_layer)
    return model_info

model_info = inject_extra_layers(model_info)


### Step 4: Recompile and Assemble
Rebuild the model using the obfuscated information

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Layer

def build_obfuscated_model(model_info):
    input_layer = Input(shape=model.input_shape[1:])
    x = input_layer
    layer_dict = {}

    for layer_info in model_info['layers']:
        LayerClass = getattr(tf.keras.layers, layer_info['class_name'])
        if layer_info['class_name'] == 'Add':
            x = Add()([x, layer_dict[layer_info['name']]])
        else:
            x = LayerClass(**layer_info['config'])(x)
            layer_dict[layer_info['name']] = x

    obfuscated_model = Model(inputs=input_layer, outputs=x)
    return obfuscated_model

obfuscated_model = build_obfuscated_model(model_info)
obfuscated_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### Test and Deploy
Test the obfuscated model to ensure it functions correctly

In [ ]:
# Assuming you have test data
test_data, test_labels = load_test_data()

# Evaluate the obfuscated model
loss, accuracy = obfuscated_model.evaluate(test_data, test_labels)
print(f'Obfuscated Model Accuracy: {accuracy * 100:.2f}%')

# Save the obfuscated model
obfuscated_model.save('path/to/obfuscated_model.h5')
